In [68]:
import pandas as pd
import numpy as np
import time

In [49]:
pp_2022 = pd.read_csv("../data/poultry_plants_2022.csv")
pp_sales = pd.read_csv("../data/poultry_plants_with_sales.csv")
fsis = pd.read_csv("../data/fsis-processors-with-location.csv")

In [52]:
no_match = pp_sales[pp_sales["Sales Volume (Location)"].isna()]
no_match.head()

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude,Sales Volume (Location)
3,74,P7342 + V7342,4516,Cargill,Wayne Farms LLC,Alabama,Large,Chicken,5.0,5.0,"808 Ross Clark Circle NE, Dothan, AL 36303",31.225754,-85.362068,NaN
4,75,P1235,4509,Cargill,Wayne Farms LLC,Alabama,Large,Chicken,5.0,5.0,"254 Ipsco Road, Decatur, AL 35601",34.607601,-87.042595,NaN
8,79,P6666,4515,Koch Foods,"Koch Foods of Gadsden, LLC",Alabama,Large,Chicken,5.0,5.0,"501 Paden Road, Gadsden, AL 35903",33.970439,-85.972439,NaN
10,81,P6529,4595,Koch Foods,"Koch Foods of Montgomery, AL",Alabama,Large,Chicken,5.0,5.0,"3500 West Blvd., Montgomery, AL 36108",32.328428,-86.354784,NaN
12,83,P6504,4496,Peco Foods,"Peco Foods, Inc.",Alabama,Large,Chicken,5.0,5.0,"3701 Reese Phifer Avenue, Tuscaloosa, AL 35401",33.177866,-87.561447,NaN


In [53]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [85]:
no_match = pp_sales[pp_sales["Sales Volume (Location)"].isna()]
len(no_match)

54

In [83]:
threshold = 5 #(km)

for index, row in no_match.iterrows():
    target_point = (row["latitude"], row["longitude"])

    for j, infogroup in pp_2022.iterrows():
        candidate_point = infogroup["LATITUDE"], infogroup["LONGITUDE"]
        distance = haversine(target_point[1], target_point[0], candidate_point[1], candidate_point[0])
        if distance <= threshold:
            print("current point: " + str(target_point) + "; match from pp22: " + str(candidate_point))
            print("current company: " + row["Parent Corporation"] + ", " + row["Establishment Name"] + "; matched: parent ABI-" + str(infogroup["PARENT NUMBER"]) + " " + infogroup["COMPANY"])
            time.sleep(2)
            x = input("confirm location")
            if (x == "yes"):
                pp_sales.loc[index, "Sales Volume (Location)"] = infogroup["SALES VOLUME (9) - LOCATION"]
                break

current point: (35.7652697, -91.641319); match from pp22: (35.764666, -91.648842)
current company: George's, Ozark Mountain Poultry, Inc.; matched: parent ABI-531052413.0 PECO FOODS INC
current point: (35.473009, -93.457503); match from pp22: (35.4904, -93.4823)
current company: Tyson, Tyson Foods, Inc.; matched: parent ABI-7537913.0 TYSON FOODS INC
current point: (36.1905228, -94.1254707); match from pp22: (36.154021, -94.154599)
current company: Tyson, Tyson Foods, Inc.; matched: parent ABI-7537913.0 BRUSS CO
current point: (36.7164767, -119.8229319); match from pp22: (36.693249, -119.783614)
current company: Foster Farms, Zorro Leasing LLC; matched: parent ABI-9564816.0 FOSTER FARMS
current point: (37.6559523, -87.5184314); match from pp22: (37.673862, -87.546565)
current company: Tyson, Tyson Foods, Inc.; matched: parent ABI-7537913.0 TYSON FOODS INC
current point: (36.7934666, -89.94466279999999); match from pp22: (36.7933, -89.9725)
current company: Tyson, Tyson Foods Inc; matche

In [84]:
pp_sales["Sales Volume (Location)"].isna().sum()

54

In [86]:
(184-54)/184 # number of matches

0.7065217391304348

In [94]:
pp_2022

,Unnamed: 0,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,...,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,11380,COOTERS FARM,1784 TUCK WILKES RD,NOXAPATER,MS,39346.0,9311.0,159.0,662,2,...,3,950200.0,4.0,33.011665,-89.126923,P,0.0,NaN,0.0,28159.0
1,21772,TYSON FOODS INC,47283 SD HIGHWAY 34,COLMAN,SD,57017.0,6543.0,101.0,605,2,...,3,959600.0,2.0,43.978463,-96.753100,P,0.0,NaN,0.0,46101.0
2,21912,SIMMONS FOODS INC,2101 TWIN CIRCLE DR,VAN BUREN,AR,72956.0,6027.0,33.0,479,2,...,7,20501.0,2.0,35.428061,-94.336725,P,22900.0,2.0,0.0,5033.0
3,39534,TYSON FOODS INC,605 235-3301 #DD813,NORTH SIOUX CITY,SD,57049.0,NaN,127.0,605,2,...,5,20300.0,2.0,42.524600,-96.497100,X,43580.0,2.0,0.0,46127.0
4,74599,MAR-JAC POULTRY INC,1301 JAMES ST,HATTIESBURG,MS,39401.0,4570.0,35.0,601,2,...,7,10500.0,1.0,31.307197,-89.278029,P,25620.0,2.0,279.0,28035.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,16677215,PILGRIMS PRIDE,2237 WHITLEY RD E,WILSON,NC,27893.0,7951.0,195.0,252,2,...,7,700.0,1.0,35.732556,-77.880145,P,48980.0,1.0,468.0,37195.0
716,16774099,2-C'S ENTERPRISES,961 WHITES BOTTOM RD,BURKESVILLE,KY,42717.0,NaN,57.0,270,2,...,5,950100.0,3.0,36.780400,-85.381500,X,0.0,NaN,0.0,21057.0
717,16797787,DAYBREAK FOODS INC,533 E TYRANENA PARK RD,LAKE MILLS,WI,53551.0,9683.0,55.0,920,2,...,5,100500.0,2.0,43.087388,-88.893628,P,48020.0,1.0,376.0,55055.0
718,16806974,BELLARD'S POULTRY,405 S BULLARD ST,OPELOUSAS,LA,70570.0,2575.0,97.0,337,2,...,7,961600.0,1.0,30.532858,-92.098679,P,36660.0,1.0,318.0,22097.0


In [87]:
pp_sales_updated = pp_sales.copy()

In [98]:
median = pp_sales.groupby(['Parent Corporation'])['Sales Volume (Location)'].median().reset_index()
median

,Parent Corporation,Sales Volume (Location)
0,Amick,NaN
1,Aterian Investment Partners,NaN
2,Bachoco,16680.0
3,Cargill,243269.0
4,Case Farms,0.0
5,Costco,NaN
6,Farmer Focus,19328.0
7,Fieldale Farms Corporation,270690.5
8,Foster Farms,972698.0
9,George's,14743.5


In [146]:
dict1 = dict(zip(median["Parent Corporation"], median["Sales Volume (Location)"]))

In [175]:
pp_sales_updated2 = pp_sales_updated.copy()

In [176]:
for index, row in pp_sales_updated2.iterrows():
    if np.isnan(row["Sales Volume (Location)"]):
        parent = row["Parent Corporation"]
        pp_sales_updated2.loc[index, "Sales Volume (Location)"] = dict1[parent]
        
pp_sales_updated2

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude,Sales Volume (Location)
0,71,P1317 + V1317,4495,Cargill,Wayne Farms LLC,Alabama,Large,Chicken,5.0,5.0,"700 McDonald Avenue, Albertville, AL 35950",34.260726,-86.203222,438268.0
1,72,P7485 + V7485,4518,Cargill,Wayne Farms LLC,Alabama,Large,Chicken,5.0,5.0,"1020 County Road 114, Jack, AL 36346",31.500628,-85.903438,576660.0
2,73,P912,4493,Cargill,"Wayne Farms, LLC",Alabama,Large,Chicken,5.0,5.0,"444 Baskin Street South, Union Springs, AL 36089",32.139166,-85.721704,271713.0
3,74,P7342 + V7342,4516,Cargill,Wayne Farms LLC,Alabama,Large,Chicken,5.0,5.0,"808 Ross Clark Circle NE, Dothan, AL 36303",31.225754,-85.362068,243269.0
4,75,P1235,4509,Cargill,Wayne Farms LLC,Alabama,Large,Chicken,5.0,5.0,"254 Ipsco Road, Decatur, AL 35601",34.607601,-87.042595,8474.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,610,P45912,6162782,Independent,Midwest Poultry Processing LLC,Missouri,Very Small,Chicken,2.0,4.0,"47422 State Hwy V, Rutledge, MO 63563",40.272931,-92.025286,1631.0
180,611,M47472 + P47472,6165638,Independent,"Pure Pasture Packing, LLC",Missouri,Very Small,Chicken,2.0,3.0,"29612 Pony Path Road, Sedalia, MO 65301",38.687938,-93.140737,46949.0
181,612,M46730 + P46730,6164667,Independent,"NGF Processing, LLC",Mississippi,Very Small,Chicken,1.0,2.0,"367-A Mckenzie Rd, Petal, MS 39465",31.237684,-89.105619,46949.0
182,617,M744 + P744,846,Independent,Vineland Poultry LLC,New Jersey,Very Small,Chicken,4.0,4.0,"1100 South Mill Road, Vineland, NJ 8360",39.471963,-75.063579,19330.0


In [179]:
pp_sales_updated2[np.isnan(pp_sales_updated2["Sales Volume (Location)"])]

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude,Sales Volume (Location)
35,111,P935,2260,Harim Group,Allen Harim LLC,Delaware,Large,Chicken,5.0,5.0,"18752 Harbeson Road, Harbeson, DE 19951",38.720742,-75.287243,NaN
67,148,P7927,2351,Amick,"AMICK FARMS, LLC",Maryland,Large,Chicken,5.0,5.0,"274 NEALSON STREET, HURLOCK, MD 21643",38.635641,-75.858116,NaN
76,161,P519,4505,Amick,Amick Farms LLC,Mississippi,Large,Chicken,5.0,5.0,"525 N Mississippi Ave, Laurel, MS 39440",31.699760,-89.118546,NaN
103,189,P48304,6163885,Costco,Lincoln Premium Poultry,Nebraska,Large,Chicken,5.0,5.0,"1325 E. Cloverly Road, Fremont, NE 68025",41.417900,-96.481808,NaN
106,193,M1015 + P1015,6413,Aterian Investment Partners,"Table Trust Brands, LLC",Pennsylvania,Large,Chicken,5.0,5.0,"247 Empire Drive, Mifflintown, PA 17059",40.558643,-77.396183,NaN
110,198,P7987,672,Amick,"Amick Farms, LLC",South Carolina,Large,Chicken,5.0,5.0,"2079 Batesburg Hwy., Batesburg, SC 29006",33.954947,-81.636345,NaN


In [142]:
pp_sales_updated2["Sales Volume (Location)"].isna().sum()

# 6 rows with no sales volume, reduced from 54 after filling in parent corporation medians

54